### [ResNet50-V1](https://medium.com/@sharma.tanish096/detailed-explanation-of-residual-network-resnet50-cnn-model-106e0ab9fa9e)

#### Import libraries


In [30]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, MaxPooling2D, GlobalAveragePooling2D, Activation, Add, BatchNormalization, ZeroPadding2D
from tensorflow.keras import Model

#### Conv_BN_Relu function

In [38]:
def conv_block(x, filters, kernel_size, strides, padding='same'):
  """
  Basic function defining sequential operations -> conv2d + BatchNorm + Relu

  x -> input layer
  """
  x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)(x)
  x = BatchNormalization()(x)
  x = Activation(activation='relu')(x)
  return x

#### Identity block

In [39]:
def identity_block(x, filters):
  """
  Used when input and output dimensions match

  [(1x1conv+BN+Relu -> 3x3conv+BN+Relu -> 1x1conv+BN) + x ] -> Relu
  """
  shortcut = x

  x = conv_block(x, filters, kernel_size=1, strides=1)
  x = conv_block(x, filters, kernel_size=3, strides=1)
  x = Conv2D(filters=filters*4, kernel_size=1)(x)
  x = BatchNormalization()(x)
  x = Add()([x, shortcut])
  x = Activation('relu')(x)
  return x


#### Convolutional/projection block

In [40]:
def convolutional_block(x, filters, strides):
  """
  Used when input and output dimensions don't match

  [(1x1conv+BN+Relu -> 3x3conv+BN+Relu -> 1x1conv+BN)  + skip(1x1conv+BN)] -> Relu
  """

  shortcut = x

  x = conv_block(x, filters, kernel_size=1, strides=strides)
  x = conv_block(x, filters, kernel_size=3, strides=1)
  x = Conv2D(filters=filters*4, kernel_size=1)(x)
  x = BatchNormalization()(x)

  shortcut = Conv2D(filters=filters*4, kernel_size=1, strides=strides)(shortcut)
  shortcut = BatchNormalization()(shortcut)

  x = Add()([x, shortcut])
  x = Activation('relu')(x)
  return x

#### Build Model

In [41]:
def Resnet50V1(input_shape=(224,224,3),num_classes=1000):
  """
  Resnet50-v1 model using tensorflow

  convolutional block -> when input and output dimension doesn't match
  identity block -> when input and output dimension matches

  Resnet50  -> blocks (3,4,6,3)
  """
  inputs = Input(shape=input_shape)

  # initial conv+maxpool layer
  x = Conv2D(filters=64, kernel_size=(7,7),strides=(2,2), padding='same')(inputs)
  x = BatchNormalization()(x)
  x = Activation(activation='relu')(x)
  x = MaxPooling2D(pool_size=(3,3),strides=(2,2), padding='same')(x)

  # block1
  x = convolutional_block(x, filters=64, strides=(1,1))
  x = identity_block(x, filters=64)
  x = identity_block(x, filters=64)

  # block2
  x = convolutional_block(x, filters=128, strides=(2,2))
  x = identity_block(x, filters=128)
  x = identity_block(x, filters=128)
  x = identity_block(x, filters=128)

  # block 3
  x = convolutional_block(x, filters=256, strides=(2,2))
  x = identity_block(x, filters=256)
  x = identity_block(x, filters=256)
  x = identity_block(x, filters=256)
  x = identity_block(x, filters=256)
  x = identity_block(x, filters=256)

  # block 4
  x = convolutional_block(x, filters=512, strides=(2,2))
  x = identity_block(x, filters=512)
  x = identity_block(x, filters=512)

  # global average pooling
  x = GlobalAveragePooling2D()(x)
  outputs = Dense(units=num_classes, activation='softmax')(x)

  # Model
  model = Model(inputs=inputs, outputs=outputs)
  return model



#### Model summary

In [42]:
model = Resnet50V1()
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_205 (Conv2D)         (None, 112, 112, 64)         9472      ['input_9[0][0]']             
                                                                                                  
 batch_normalization_206 (B  (None, 112, 112, 64)         256       ['conv2d_205[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_184 (Activation  (None, 112, 112, 64)         0         ['batch_normalization_20